In [1]:
import pickle
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import math
import time
import re
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity  
from sklearn.metrics import pairwise_distances
from matplotlib import gridspec
from scipy.sparse import hstack
import plotly
import plotly.figure_factory as ff
from plotly.graph_objs import Scatter, Layout

plotly.offline.init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")


#### Remove near duplicate items

In [2]:
#Understand about duplicates
# read data from pickle file from previous stage
data=pd.read_pickle('Pickle/28k_apparel_data')

In [3]:
# Finding Number of products that have duplicate titles.
sum(data.duplicated ('title'))
## we have 2325 products which have same title but different color

2325

In [4]:
pip install notebook --upgrade


Requirement already up-to-date: notebook in /Users/Samrin/anaconda3/lib/python3.7/site-packages (6.0.2)
Note: you may need to restart the kernel to use updated packages.


#### These shirts are exactly same except in size (S, M,L,XL)

#### These shirts exactly same except in color

#### In our data there are many duplicate products similar to the above examples, we need to de-dupe them for better outcomes.

#### Remove duplicates : Part 1

In [5]:
#  Dta from pickle file from previous stage
data=pd.read_pickle('Pickle/28k_apparel_data')
data.head()

asin                        brand              color  \
4   B004GSI2OS                  FeatherLite  Onyx Black/ Stone   
6   B012YX2ZPI  HX-Kingdom Fashion T-shirts              White   
11  B001LOUGE4                 Fitness Etc.              Black   
15  B003BSRPB0                  FeatherLite              White   
21  B014ICEDNA                        FNC7C             Purple   

                                     medium_image_url product_type_name  \
4   https://images-na.ssl-images-amazon.com/images...             SHIRT   
6   https://images-na.ssl-images-amazon.com/images...             SHIRT   
11  https://images-na.ssl-images-amazon.com/images...             SHIRT   
15  https://images-na.ssl-images-amazon.com/images...             SHIRT   
21  https://images-na.ssl-images-amazon.com/images...             SHIRT   

                                                title formatted_price  
4   Featherlite Ladies' Long Sleeve Stain Resistan...          $26.26  
6   Women's Unique 100% Cotton T - Special Olympic...           $9.99  
11             Ladies Cotton Tank 2x1 Ribbed Tank Top          $11.99  
15  FeatherLite Ladies' Moisture Free Mesh Sport S...          $20.54  
21  Supernatural Chibis Sam Dean And Castiel Short...           $7.50

In [6]:
# Reamove all the products with very few descriptive words in title
#data_sorted = data[data['title'].apply(lambda x: len(x.split())>4)]
data_sorted=data[data['title'].apply(lambda x : len (x.split()) > 4)]
print("After removal of products with short description:", data_sorted.shape[0])

After removal of products with short description: 27949


In [7]:
# Sort the whole data based on title (alphabetical order of title) 
data_sorted.sort_values('title',inplace=True, ascending=False)
data_sorted.head(20)


asin             brand                     color  \
61973   B06Y1KZ2WB            Éclair                Black/Pink   
133820  B010RV33VE          xiaoming                      Pink   
81461   B01DDSDLNS          xiaoming                     White   
75995   B00X5LYO9Y          xiaoming               Red Anchors   
151570  B00WPJG35K          xiaoming                     White   
152506  B00WOB0LHK          xiaoming                  Red+Grey   
76209   B014GZY5KK          xiaoming              Figure Color   
103647  B00WPJHEE4          xiaoming                    Yellow   
138588  B00X5MLMRA          xiaoming                    Yellow   
95954   B00WPK4T4G          xiaoming                 red+whtie   
74338   B00X9Z53FU          xiaoming                      Blue   
18905   B0711JTVXY       Xhilaration           Purple/Raccoons   
60078   B0722WJ5H3       Xhilaration           Bubble Gum Pink   
59030   B071VKWKLV       Xhilaration           Bubble Gum Pink   
177229  B0756RHHHV              Have  Mustard Yellow and Black   
22273   B074741CW9       White Horse                     Black   
135865  B01I7ZZYE6  heather trujillo                     White   
171318  B01I80YZNG  heather trujillo                      Grey   
44082   B01MPYKVER       White Horse                 Turquoise   
40331   B071JRW7BN        wallpapHER                     Black   

                                         medium_image_url  \
61973   https://images-na.ssl-images-amazon.com/images...   
133820  https://images-na.ssl-images-amazon.com/images...   
81461   https://images-na.ssl-images-amazon.com/images...   
75995   https://images-na.ssl-images-amazon.com/images...   
151570  https://images-na.ssl-images-amazon.com/images...   
152506  https://images-na.ssl-images-amazon.com/images...   
76209   https://images-na.ssl-images-amazon.com/images...   
103647  https://images-na.ssl-images-amazon.com/images...   
138588  https://images-na.ssl-images-amazon.com/images...   
95954   https://images-na.ssl-images-amazon.com/images...   
74338   https://images-na.ssl-images-amazon.com/images...   
18905   https://images-na.ssl-images-amazon.com/images...   
60078   https://images-na.ssl-images-amazon.com/images...   
59030   https://images-na.ssl-images-amazon.com/images...   
177229  https://images-na.ssl-images-amazon.com/images...   
22273   https://images-na.ssl-images-amazon.com/images...   
135865  https://images-na.ssl-images-amazon.com/images...   
171318  https://images-na.ssl-images-amazon.com/images...   
44082   https://images-na.ssl-images-amazon.com/images...   
40331   https://images-na.ssl-images-amazon.com/images...   

           product_type_name  \
61973                  SHIRT   
133820                 SHIRT   
81461                  SHIRT   
75995                  SHIRT   
151570                 SHIRT   
152506                 SHIRT   
76209                  SHIRT   
103647                 SHIRT   
138588                 SHIRT   
95954                  SHIRT   
74338                  SHIRT   
18905                  SHIRT   
60078                  SHIRT   
59030                  SHIRT   
177229                 SHIRT   
22273                  SHIRT   
135865  BOOKS_1973_AND_LATER   
171318  BOOKS_1973_AND_LATER   
44082                  SHIRT   
40331                  SHIRT   

                                                    title formatted_price  
61973   Éclair Women's Printed Thin Strap Blouse Black...          $24.99  
133820  xiaoming Womens Sleeveless Loose Long T-shirts...          $18.19  
81461   xiaoming Women's White Long Sleeve Single Brea...          $21.58  
75995   xiaoming Stripes Tank Patch/Bear Sleeve Anchor...          $15.91  
151570  xiaoming Sleeve Sheer Loose Tassel Kimono Woma...          $14.32  
152506  xiaoming Sleeve Plaid Checked Casual Tops Shir...          $15.38  
76209   xiaoming Shirt Black Blouse Casual Oversized W...          $26.15  
103647  xiaoming Casual Women Long Loose Sleeve Shirt ...       

#### Some examples of dupliacte titles that differ only in the last few words.

Titles 1:
16. woman's place is in the house and the senate shirts for Womens XXL White
17. woman's place is in the house and the senate shirts for Womens M Grey

Title 2:
25. tokidoki The Queen of Diamonds Women's Shirt X-Large
26. tokidoki The Queen of Diamonds Women's Shirt Small
27. tokidoki The Queen of Diamonds Women's Shirt Large

Title 3:
61. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
62. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
63. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt
64. psychedelic colorful Howling Galaxy Wolf T-shirt/Colorful Rainbow Animal Print Head Shirt for woman Neon Wolf t-shirt


In [8]:
indices=[]
for i, row in data_sorted.iterrows():
    indices.append(i)

In [9]:
import itertools
stage1_dedupe_asins = []
i = 0
j = 0
num_data_points = data_sorted.shape[0]
while i < num_data_points and j < num_data_points:
    
    previous_i = i

    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    a = data['title'].loc[indices[i]].split()

    # search for the similar products sequentially 
    j = i+1
    while j < num_data_points:

        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'Small']
        b = data['title'].loc[indices[j]].split()

        # store the maximum length of two strings
        length = max(len(a), len(b))

        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0] == k[1]):
                count += 1

        # if the number of words in which both strings differ are > 2 , we are considering it as those two apperals are different
        # if the number of words in which both strings differ are < 2 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) > 2: # number of words in which both sensences differ
            # if both strings are differ by more than 2 words we include the 1st string index
            stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[i]])

            # if the comaprision between is between num_data_points, num_data_points-1 strings and they differ in more than 2 words we include both
            if j == num_data_points-1: stage1_dedupe_asins.append(data_sorted['asin'].loc[indices[j]])

            # start searching for similar apperals corresponds 2nd string
            i = j
            break
        else:
            j += 1
    if previous_i == i:
        break

In [10]:
data = data.loc[data['asin'].isin(stage1_dedupe_asins)]

#### We removed the dupliactes which differ only at the end of the titles as the examoples seen above.

In [11]:
print('Number of data points:' , data.shape[0])

Number of data points: 17593


In [12]:
data=pd.read_pickle('Pickle/17k_apperal_data')

#### Remove duplicates : Part 2

In the previous cell, we sorted whole data in alphabetical order of  titles.Then, we removed titles which are adjacent and very similar title

But there are some products whose titles are not adjacent but very similar.

Examples:

Titles-1
86261.  UltraClub Women's Classic Wrinkle-Free Long Sleeve Oxford Shirt, Pink, XX-Large
115042. UltraClub Ladies Classic Wrinkle-Free Long-Sleeve Oxford Light Blue XXL

TItles-2
75004.  EVALY Women's Cool University Of UTAH 3/4 Sleeve Raglan Tee
109225. EVALY Women's Unique University Of UTAH 3/4 Sleeve Raglan Tees
120832. EVALY Women's New University Of UTAH 3/4-Sleeve Raglan Tshirt


In [19]:
data = pd.read_pickle('Pickle/17k_apperal_data')

In [ ]:
indices = []
for i,row in data.iterrows():
    indices.append(i)

stage2_dedupe_asins = []
while len(indices)!=0:
    i = indices.pop()
    stage2_dedupe_asins.append(data['asin'].loc[i])
    # consider the first apperal's title
    a = data['title'].loc[i].split()
    # store the list of words of ith string in a, ex: a = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
    for j in indices:
        
        b = data['title'].loc[j].split()
        # store the list of words of jth string in b, ex: b = ['tokidoki', 'The', 'Queen', 'of', 'Diamonds', 'Women's', 'Shirt', 'X-Large']
        
        length = max(len(a),len(b))
        
        # count is used to store the number of words that are matched in both strings
        count  = 0

        # itertools.zip_longest(a,b): will map the corresponding words in both strings, it will appened None in case of unequal strings
        # example: a =['a', 'b', 'c', 'd']
        # b = ['a', 'b', 'd']
        # itertools.zip_longest(a,b): will give [('a','a'), ('b','b'), ('c','d'), ('d', None)]
        for k in itertools.zip_longest(a,b): 
            if (k[0]==k[1]):
                count += 1

        # if the number of words in which both strings differ are < 3 , we are considering it as those two apperals are same, hence we are ignoring them
        if (length - count) < 3:
            indices.remove(j)

In [15]:
# Taking in account only the products that are found in previous cell 
data=data.loc[data['asin'].isin(stage2_dedupe_asins)]

#### 17k apperals we reduced to 16k apperals

In [16]:
print('Number of data points after stage two of dedupe: ',data.shape[0])

Number of data points after stage two of dedupe:  16435


In [20]:
data.to_pickle('Pickle/16k_apperal_data')

_End_